# Imports

In [1]:
import numpy as np
import pandas as pd
import random
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import os

import sys
sys.path.insert(1, './../src/')

import time 

import load_option_data_01 

from pathlib import Path

import bsm_pricer as bsm
import config
import datetime
import level_1_filters as f1
import level_2_filters as f2
import level_3_filters as f3
import load_option_data_01 
import load_option_data_01 as l1


import time 
import warnings
import wrds

from scipy.stats import norm
from scipy.spatial.distance import cdist

In [2]:
import importlib

In [3]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pio.templates.default = "plotly_white"
warnings.filterwarnings("ignore")

In [4]:
OUTPUT_DIR = Path(config.OUTPUT_DIR)
DATA_DIR = Path(config.DATA_DIR)
WRDS_USERNAME = config.WRDS_USERNAME

START_DATE_01 =config.START_DATE_01
END_DATE_01 = config.END_DATE_01

START_DATE_02 =config.START_DATE_02
END_DATE_02 = config.END_DATE_02

NOTE_START = START_DATE_01
NOTE_END = END_DATE_01

In [5]:
DATE_RANGE =f'{pd.Timestamp(START_DATE_01):%Y-%m}_{pd.Timestamp(END_DATE_02):%Y-%m}'

# Functions

In [6]:
# --- Black-Scholes elasticity ---
def bs_elasticity(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d1 - sigma * np.sqrt(T))
        delta = norm.cdf(d1)
    else:
        price = K * np.exp(-r * T) * norm.cdf(-d1 + sigma * np.sqrt(T)) - S * norm.cdf(-d1)
        delta = -norm.cdf(-d1)
    return (delta * S / price), price

# --- Kernel Weights ---
def kernel_weights(moneyness_grid, maturity_grid, k_s, ttm, bw_m=0.0125, bw_t=10):
    grid = np.column_stack((moneyness_grid, maturity_grid))
    point = np.array([[k_s, ttm]])
    dists = cdist(grid, point, 'euclidean')[:, 0]
    weights = np.exp(-0.5 * (dists / np.array([bw_m, bw_t])).sum())
    return weights / weights.sum()

# --- Construct a single day portfolio ---
def construct_portfolio(data, k_s_target, ttm_target, option_type='call', r=0.01):
    subset = data[(data['option_type'] == option_type)]
    weights = kernel_weights(subset['moneyness'], subset['ttm'], k_s_target, ttm_target)
    subset = subset.assign(weight=weights)
    subset = subset[subset['weight'] > 0.01]
    subset['weight'] /= subset['weight'].sum()

    # Leverage-adjusted returns
    elast, price = bs_elasticity(
        S=subset['underlying'], K=subset['strike'], T=subset['ttm']/365,
        r=r, sigma=subset['iv'], option_type=option_type
    )
    subset['leverage_return'] = subset['daily_return'] / elast

    return (subset['leverage_return'] * subset['weight']).sum()

# --- Main Loop (simplified) ---
def build_portfolios(option_data, m_grid, ttm_grid, option_types=['call', 'put']):
    portfolios = []
    for opt_type in option_types:
        for k_s in m_grid:
            for ttm in ttm_grid:
                ret = construct_portfolio(option_data, k_s, ttm, option_type=opt_type)
                portfolios.append({
                    'type': opt_type,
                    'moneyness': k_s,
                    'ttm': ttm,
                    'return': ret
                })
    return pd.DataFrame(portfolios)


In [7]:
# read filtered data

spx_filtered = pd.read_parquet(Path(DATA_DIR / f'spx_filtered_final_{DATE_RANGE}.parquet'))
spx_filtered

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\viren\\Documents\\GitHub\\constantinides_2013_options\\data\\spx_filtered_final_1996-01_2019-12.parquet'

## Portfolio Construction Process

Portfolios are defined as follows:
- Portfolio return from $t-1$ to $t$ is defined as $\Rho_{C or P,m,T,t}$, where $\Rho$ is the daily return of a call ($C$) or put ($P$) portfolio, with moneyness $m$ and maturity $T$ days, where $T \in {30, 60, 90}$, at time $t$.
$$
\begin{align}
\Rho_{C,m,t} &= xC_t - yr_f \\
\Rho_{P,m,t} &= -zP_t + br_f

\end{align}
$$


In [ ]:
pd.date_tDATE_RANGE

In [ ]:
moneyness_tgts = [0.90, 0.925, 0.95, 0.975, 1.00, 1.025, 1.05, 1.075, 1.10] # K/S, so < 1 = puts are OTM/calls are ITM, and > 1 calls are OTM/puts are ITM
maturity_tgts = [30, 60, 90]  # in days. For HKM, these are averaged
option_types = ['C', 'P']  # 'call' and 'put' options

portfolio_idx = pd.MultiIndex.from_tuples(
    [(opt_type, k_s, ttm) for opt_type in option_types for k_s in moneyness_tgts for ttm in maturity_tgts for trade_date in spx_filtered.index.get_level_values('date')],
    names=['option_type', 'moneyness', 'days_to_maturity', 'trade_date']
)

portfolios = pd.DataFrame(index=portfolio_idx, columns=['daily_rets', 'monthly_rets'])
portfolios

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000001F7B5548090>>
Traceback (most recent call last):
  File "c:\Users\viren.desai\OneDrive - Aprio, LLP\Documents\repos\constantinides_2013_options\puzzle\Lib\site-packages\ipykernel\ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
